# Simple IEX Stock Fetcher



## Test api

In [1]:
import iex
import key
iex.init(key.test_token, api='sandbox')
aapl = iex.Stock('AAPL')
aapl.chart('5d', chartCloseOnly=True)

SyntaxError: invalid syntax (iex.py, line 25)

In [2]:
aapl.dividends('1y')

,paymentDate,recordDate,declaredDate,amount,flag,currency,description,frequency,date
exDate,,,,,,,,,
2019-11-17,2019-11-23,2019-11-16,2019-11-01,0.79,Chas,USD,Od Seyhrnrsiaar,reqrltauy,2019-12-07
2019-08-10,2019-08-22,2019-08-21,2019-08-07,0.78,aCsh,USD,irraSyn ershOad,yaruqtrle,2019-12-07
2019-05-20,2019-05-29,2019-05-17,2019-05-13,0.81,hasC,USD,s eahndrSyrrOai,tarleyuqr,2019-12-07
2019-02-11,2019-02-17,2019-02-22,2019-02-01,0.74,ahsC,USD,raahedyOsn rriS,erqulytar,2019-12-07


In [14]:
# iex.init(key.token, api='cloud')
# aapl.dividends('1y')

,paymentDate,recordDate,declaredDate,amount,flag,currency,description,frequency,date
exDate,,,,,,,,,
2019-11-07,2019-11-14,2019-11-11,2019-10-30,0.77,Cash,USD,Ordinary Shares,quarterly,2019-12-07
2019-08-09,2019-08-15,2019-08-12,2019-07-30,0.77,Cash,USD,Ordinary Shares,quarterly,2019-12-07
2019-05-10,2019-05-16,2019-05-13,2019-04-30,0.77,Cash,USD,Ordinary Shares,quarterly,2019-12-07
2019-02-08,2019-02-14,2019-02-11,2019-01-29,0.73,Cash,USD,Ordinary Shares,quarterly,2019-12-07


In [2]:
iex.Stock('AIRT').splits('5y')

,declaredDate,ratio,toFactor,fromFactor,description,date
exDate,,,,,,
2019-06-18,2019-06-13,0.673858,3,2,i-rp2fS t3-ol,2019-12-07
2019-06-20,2019-06-08,0.672697,3,2,ptoiSr- 23-lf,2019-12-07


# Fetch Symobols

In [5]:
import pandas as pd
# 这东西得每天都可以更新下
symbols_path = "../../historical_data/us/prices/iex/symbols.csv"
old_symbols = pd.read_csv(symbols_path, index_col='date')

iex.init(key.token, api='cloud')
symbols = iex.Reference.symbols()
iex.init(key.test_token, api='sandbox')

symbols = pd.concat([old_symbols, symbols], sort=False)
symbols.to_csv(symbols_path)

type	refers to the common issue type

    ad - ADR
    re - REIT
    ce - Closed end fund
    si - Secondary Issue
    lp - Limited Partnerships
    cs - Common Stock
    et - ETF
    wt - Warrant
    oef - Open Ended Fund
    cef - Closed Ended Fund
    ps - Preferred Stock
    ut - Unit
    struct - Structured Product

In [9]:
symbols.loc[symbols.index[-1]].groupby('type').symbol.count()

type
ad         403
cef        505
cs        4534
et        2340
ps         762
rt          22
struct       2
temp         2
ut         116
wt         178
Name: symbol, dtype: int64

# Fetch Sectors

In [3]:
# 获取保存行业列表
iex.init(key.token, api='cloud')
iex_path = "../../historical_data/us/prices/iex/"
df = iex.Reference.sectors()
df.to_csv(iex_path + 'sectors.csv')
df = iex.Reference.tags()
df.to_csv(iex_path + 'tags.csv')

### Fetch Companys

In [1]:
import pandas as pd
import numpy as np
symbols_path = "../../historical_data/us/prices/iex/symbols.csv"
symbols = pd.read_csv(symbols_path, index_col='date', parse_dates=True)
# select last date
symbols = symbols.loc[symbols.index[-1]]
print(symbols.index[0])

symbols = symbols[(symbols.type == 'ad') | (symbols.type == 'cs') & (symbols.exchange != 'OTC')]
symbols = symbols.symbol.values
symbols = np.append(symbols, ['SPY', 'QQQ'])
symbols, symbols.shape

2019-12-20 00:00:00


(array(['A', 'AA', 'AACG', ..., 'ZYXI', 'SPY', 'QQQ'], dtype=object), (4938,))

In [2]:
import iex
import key
from tqdm.notebook import tqdm
iex_path = "../../historical_data/us/prices/iex/"

# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')

old_companys = pd.read_csv(iex_path + 'companys.csv', index_col='symbol')
symbols = set(symbols) - set(old_companys.index)

companys_list = []
for k in tqdm(symbols):
    try:
        companys_list.append(iex.Stock(k).company())
    except ConnectionError as e:
        print(e)
    
companys = pd.DataFrame(companys_list)
companys = companys.set_index('symbol')
old_companys = old_companys[~old_companys.index.isin(companys.index)]
companys = pd.concat([old_companys, companys], sort=False)
companys.to_csv(iex_path + 'companys.csv')

# iex的数据问题
split有很多重复数据，且部分股票比如TR完全没split数据，虽然他分的少100-103

# Fetcher Functions

In [10]:
import os, sys
import pandas as pd
chart_range = [(5,'5d'), (20,'1m'), (75,'3m'), (165,'6m'), (341,'1y'), (715,'2y'), (99999, '5y')]

def range_since_last(df, update_date_col):
    if update_date_col is None:
        update_at = df.index[-1]
    else:
        update_at = df[update_date_col].iloc[-1]
    update_at = pd.Timestamp(update_at)
    days = pd.Timedelta(pd.datetime.now() - update_at).days
    bdays = len(pd.bdate_range(update_at, pd.datetime.now())) - 1
    if bdays <= 0:
        return None
    for range_, param in chart_range:
        if days < range_:
            return param
    return '5y'

def fetch_on_demand(filename, index_col, update_date_col, fetch_call):
    if os.path.exists(filename):
        df = pd.read_csv(filename, index_col=index_col, parse_dates=True)
        param = range_since_last(df, update_date_col)
        sys.stdout.write(str(param) + "      ")
        if param is None:
            return
        # fetcher missing period
        try:
            df_append = fetch_call(param)
        except EOFError:  # empty data
            # update last fetch date only
            if update_date_col is not None:
                df.loc[df.index[-1], update_date_col] = pd.datetime.now()
                df.to_csv(filename)
            return
        # 注意，必须小心的处理duplication，因为div的dup是相加的
        # 首先drop_duplicates不考虑index, 所以日期不同的内容相同会被drop
        # df = df_append.combine_first(df) 对有重复数据的分红不行，会重复行
        # 所以简单的cancat然后drop
        df = pd.concat([df, df_append], sort=False)
        df = df.reset_index().drop_duplicates()
        df = df.set_index(index_col)
        df.sort_index(inplace=True)
        df.to_csv(filename)
    else:
        sys.stdout.write('new 5y     ')
        try:
            df = fetch_call('5y')
        except EOFError:  # empty data
            # save last fetch date to csv
            if update_date_col is not None:
                pd.DataFrame([[np.nan, pd.datetime.now()]], columns=[index_col, update_date_col]).to_csv(
                    filename, index=False)
            return
        df.sort_index(inplace=True)
        df.to_csv(filename)

## Fetch Adjustments

In [36]:
import numpy as np
symbols_path = "../../historical_data/us/prices/iex/symbols.csv"
symbols = pd.read_csv(symbols_path, index_col='date', parse_dates=True)
# select last date
symbols = symbols.loc[symbols.index[-1]]
print(symbols.index[0])

symbols = symbols[(symbols.type == 'ad') | (symbols.type == 'cs') & (symbols.exchange != 'OTC')]
symbols = symbols.symbol.values
symbols = np.append(symbols, ['SPY', 'QQQ'])
symbols, symbols.shape

2019-12-11 00:00:00


(array(['A', 'AA', 'AACG', ..., 'ZYXI', 'SPY', 'QQQ'], dtype=object), (4939,))

In [7]:
from tqdm.notebook import tqdm

# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')
 
for k in tqdm(symbols):
    sys.stdout.write('\r{}   '.format(k))
    filename = "./dividends/{}.csv".format(k)
    fetch_on_demand(filename, 'exDate', 'date', iex.Stock(k).dividends)
print('ok')

QQQ   new 5y        
ok


In [6]:
from tqdm.notebook import tqdm

# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')

for k in tqdm(symbols):
    sys.stdout.write('\r{}   '.format(k))
    filename = "./splits/{}.csv".format(k)
    fetch_on_demand(filename, 'exDate', 'date', iex.Stock(k).splits)
print('ok')

QQQ   None         ne      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      None      
ok


iex获取的数据还是有错，比如AIRT，要yahoo交叉验证

## Fetch stocks

In [21]:
import pandas as pd
symbols = pd.read_csv('top550.csv', header=None)
symbols = symbols[1].values
symbols.shape

(870,)

In [24]:
from tqdm.notebook import tqdm

# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')
 
for k in tqdm(symbols):
    sys.stdout.write('\r{}   '.format(k))
    filename = "./daily/{}.csv".format(k)
    def _chart(param):
        return iex.Stock(k).chart(param, changeFromClose=True)
    fetch_on_demand(filename, 'date', None, _chart)
print('ok')

PGR   new 5y       

Exception: GET https://cloud.iexapis.com/stable/stock/PGR/chart/5y?changeFromClose=True&token=pk_48abaf9c2c4f45d9adddf765ca12e400 402 You have exceeded your allotted message quota. Please enable pay-as-you-go to regain access

In [23]:
iex.init(key.test_token, api='sandbox')
# iex.init(key.token, api='cloud')

# save historical price to disk
for k in symbols:
    k = correction[k] if k in correction else k
    if k =='skip..': continue
    sys.stdout.write('\r{}   '.format(k))
    filename = "./daily/{}.csv".format(k)
    fetch_on_demand
    
    if os.path.exists(filename):
        # fetcher missing period
        df = pd.read_csv(filename, index_col='date', parse_dates=True)
        days = pd.Timedelta(pd.datetime.now() - df.index[-1]).days
        bdays = len(pd.bdate_range(df.index[-1], pd.datetime.now())) - 1
        for day, param in chart_range:
            if bdays <= 0:
                break
            elif days < day:
                print(param)
                df_append = iex.Stock(k).chart(param)
                pd.concat([df, df_append], sort=True).to_csv(filename)
                break
    else:
        print('full')
        iex.Stock(k).chart('5y').to_csv(filename)

In [4]:
# iex.init(key.token, api='cloud')
# iex.Stock('STI').chart('5y').to_csv('STI.csv')

In [18]:
import pandas as pd
import numpy as np
df = pd.read_csv('BBT.csv', index_col='date', parse_dates=True)
df = df[['uClose','uOpen','uHigh','uLow','uVolume']]
df = df.rename(columns={'uClose':'Close','uOpen':'Open','uHigh':'High','uLow':'Low','uVolume':'Volume'})
df

,Close,Open,High,Low,Volume
date,,,,,
2014-12-08,38.76,38.77,38.99,38.48,3453437
2014-12-09,38.93,38.39,38.97,38.24,5108987
2014-12-10,38.10,38.71,38.98,38.08,3834101
2014-12-11,38.18,38.16,38.54,38.11,3061449
2014-12-12,37.45,37.94,38.14,37.45,3359933
...,...,...,...,...,...
2019-12-02,54.41,54.99,55.06,54.32,3293946
2019-12-03,53.21,53.70,53.70,52.77,4541849
2019-12-04,53.31,53.25,53.51,53.07,5437363


In [ ]:
etfs = pd.read_html(requests.get('https://etfdailynews.com/etf/spy/', headers={'User-agent': 'Mozilla/5.0'}).text,
             attrs={'id': 'etfs-that-own'})
spy = [x for x in etfs[0].Symbol.values.tolist() if isinstance(x, str)]